In [1]:
import numpy as np
import scipy
import scipy.stats as sps
import pandas as pd
import matplotlib.pyplot as plt

# Homework 2
## Hypothesis testing

Each task is worth 5 points, 20 points in total. Additionally, some tasks allow you to get a bonus point, that **can not** increase your total score for the homework beyond the maximum, but can compensate for some occasionally lost points.

### Task 1: multiple comparisons

A randomized, double-blind experiment was conducted to assess the
effectiveness of several drugs for reducing postoperative nausea. The
data are as follows:

In [2]:
df1 = pd.DataFrame({'Drug': ['Placebo', 'Chlorpromazine', 'Dimenhydrinate', 'Pentobarbital (100 mg)', 'Pentobarbital (150 mg)'],
                    'Number of Patients': [80, 75, 85, 67, 85],
                    'Incidence of Nausea': [45, 26, 52, 35, 37]})
df1

,Drug,Number of Patients,Incidence of Nausea
0,Placebo,80,45
1,Chlorpromazine,75,26
2,Dimenhydrinate,85,52
3,Pentobarbital (100 mg),67,35
4,Pentobarbital (150 mg),85,37


1. Test each drug versus the placebo at the 5% level. Also, report
the estimated odds–ratios. Summarize your findings. (2 points)
2. Use the Bonferroni and the FDR method to adjust for multiple
testing. (Beecher (1959)) (3 points)
3. Reproduce plot similar to Figure 10.6 from the book, displaying observed $p$-values and different thresholds used (1 bonus point)

*Hint*. Use simple $H_0$: "$p_{drug} = p_{placebo}$".

#### Q1.1

Similar to Mendel's peas case. Hence, Pearson's Chi-squared test used:

In [3]:
# Chlorpromazine:
from scipy.stats import chi2_contingency 
   
data = [[45, 26], [35, 49]] 
stat, p, dof, expected = chi2_contingency(data) 
  
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
    print('reject the null hypothesis at the 5% level') 
else: 
    print('fail to reject the null hypothesis under the 5% level')

odds_ratio = (45/35)/(26/49)
print ('odds-ratio is', odds_ratio)

p1 = str(p)

p value is 0.011279921768145679
reject the null hypothesis at the 5% level
odds-ratio is 2.423076923076923


In [4]:
# Dimenhydrinate:
from scipy.stats import chi2_contingency 
   
data = [[45, 52], [35, 33]] 
stat, p, dof, expected = chi2_contingency(data) 
  
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
    print('reject the null hypothesis at the 5% level') 
else: 
    print('fail to reject the null hypothesis under the 5% level')
    
odds_ratio = (45/35)/(52/33)
print ('odds-ratio is', odds_ratio)

p2 = str(p)

p value is 0.6281776400799568
fail to reject the null hypothesis under the 5% level
odds-ratio is 0.815934065934066


In [5]:
# Pentobarbital (100 mg):
from scipy.stats import chi2_contingency 
   
data = [[45, 35], [35, 32]] 
stat, p, dof, expected = chi2_contingency(data) 
  
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
    print('reject the null hypothesis at the 5% level') 
else: 
    print('fail to reject the null hypothesis under the 5% level')
    
odds_ratio = (45/35)/(35/32)
print ('odds-ratio is', odds_ratio)

p3 = str(p)

p value is 0.7489122792634635
fail to reject the null hypothesis under the 5% level
odds-ratio is 1.1755102040816328


In [6]:
# Pentobarbital (150 mg):
from scipy.stats import chi2_contingency 
   
data = [[45, 37], [35, 48]] 
stat, p, dof, expected = chi2_contingency(data) 
  
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
    print('reject the null hypothesis at the 5% level') 
else: 
    print('fail to reject the null hypothesis under the 5% level')
    
odds_ratio = (45/35)/(37/48)
print ('odds-ratio is', odds_ratio)

p4 = str(p)

p value is 0.1395431162972681
fail to reject the null hypothesis under the 5% level
odds-ratio is 1.667953667953668


Only for Chlorpromazine the null hypothesis was rejected at the 5% level. Fot the other types of drugs failed to reject the null hypothesis.

#### Q1.2

In [7]:
# Holm - Bonferroni method:
p1 = float(p1)
p2 = float(p2)
p3 = float(p3)
p4 = float(p4)
p_values = [p1, p2, p3, p4]
p_sorted = sorted (p_values)

In [8]:
counter = 0
for i in range(4):
    if p_sorted[i] <= alpha/(4-i):
        counter = counter + 1
            
print(counter)

1


$\implies$ don't reject the null hypothesis for p2 and stop. The result is Reject $H_{0(1)}$ and fail to reject $H_{0(2)}$, $H_{0(3)}$ , $H_{0(4)}$.

OR:

In [9]:
# Bonferroni and Benjamini-Hochberg methods
from statsmodels.stats.multitest import multipletests
for method in ["bonferroni", "fdr_bh"]:
    reject, pvals, _, _ = multipletests(p_values, method=method, returnsorted=True)
    print(f"Method {method:10s}: reject=\n{reject}\n with p-values = \n{pvals}")

Method bonferroni: reject=
[ True False False False]
 with p-values = 
[0.04511969 0.55817247 1.         1.        ]
Method fdr_bh    : reject=
[ True False False False]
 with p-values = 
[0.04511969 0.27908623 0.74891228 0.74891228]


$\implies$ Reject $H_{0(1)}$ and fail to reject $H_{0(2)}$, $H_{0(3)}$ , $H_{0(4)}$.

### Task 2: permutation test

For this task we will use the famous Iris dataset, originaly studied by R.A. Fisher himself.

In [10]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


We will select two species: *setosa* and *virginica* and study the sepal length.

In [11]:
X = iris[iris.species == 'setosa']['sepal_width'].values
Y = iris[iris.species == 'virginica']['sepal_width'].values

Test the hypothesis $H_0$: "quantiles of level 0.2 of both flowers are equal" vs $H_1$: "quantile of level 0.2 (20 percentile, lower 20%) of the sepal width of *setosa* flowers is  larger than that of *virginica* flowers". Use permutation test, approximate the full permutation distribution with 10,000 samples. (5 points)

In [12]:
Xp = np.quantile(X, 0.2)
Yp = np.quantile(Y, 0.2)
T = np.abs(Xp-Yp)
T

0.3999999999999999

In [13]:
def approximate_permutation_criteria(func, sample1, sample2, nruns):
    l1 = len(sample1)
    l2 = len(sample2)
    
    t_obs = func(sample1, sample2)
    sample_joint = np.r_[sample1, sample2]
    counter = 0.
    for r in range(nruns):
        sample_joint = np.random.permutation(sample_joint)
        t = func(sample_joint[:l1], sample_joint[l1:])
        counter += t > t_obs    
    return counter / nruns

In [14]:
T = lambda sample1, sample2: np.abs(np.quantile(X, 0.2)-np.quantile(sample2, 0.2))

pvalue = approximate_permutation_criteria(T, X, Y, nruns=10000)
f"Got p-value = {pvalue:.3f}"

'Got p-value = 0.000'

Hence, reject the null hypothesis under any significance level.

### Task 3: computational approach to hypothesis testing

Recommended reading: http://allendowney.blogspot.com/2016/06/there-is-still-only-one-test.html

Consider the following dataset (service hours between failures of the air-conditioning equipment in a Boeing 720 jet aircraft , Proschan, 1963):

In [15]:
Y = np.array([3, 5, 7, 18, 43, 85, 91, 98, 100, 130, 230, 487])

This sample was sorted for easier presentation.

1. Compute an estimate of the median time between failures (1 point)
2. Consider the null hypothesis $H_0$ to be: "median time between failures is one week" (1 point)
3. What family of distributions will you choose for this kind of data under $H_0$? (1 point)
4. Choose a test statistic to measure the deviation from $H_0$ (1 point)
5. Check whether you can reject $H_0$ at significance level 5% and calculate the corresponding approximate $p$-value. Use 10,000 simulations for your experiment (1 point)
6. Plot histogram of the simulated values of the test statistic and mark the observed value and threshold that you obtained (1 bouns point)

In [16]:
Yw = np.array([3/168, 5/168, 7/168, 18/168, 43/168, 85/168, 91/168, 98/168, 100/168, 130/168, 230/168, 487/168])
medianweek = np.median(Yw)

$H_0: medianweek = 1$

Bernoulli distribution: each observation has one of two outcomes - it lies above or below the choosen median. Probability that lies above = probability that lies below = 0.5. Hence, S (the count of deviations from the median) follows a binomial distribution.

In [17]:
import scipy.stats as sps
from math import sqrt
S = 10
n = 12
Z = (S - 0.5*n)/(0.5*sqrt(n))
Z

2.3094010767585034

In [18]:
pvalue = 1 - sps.norm.cdf(Z)
pvalue

0.010460667668896972

2.309 > 1.96 and pvalue is less than alpha = 0.05 $\implies$ reject the null hypothesis: medianweek is different than 1.

In [19]:
def p_value(delta, data, iterations):
    count = 0.0
    for i in range(iterations):
        if np.median(data)==delta:
            count += 1
    return count/iterations

In [20]:
p_value (1, Yw, 10000)

0.0

$\implies$ reject the null hypothesis.

### Task 4: power analysis

In 1861, 10 essays appeared in the New Orleans Daily Crescent. They
were signed “Quintus Curtius Snodgrass” and some people suspected
they were actually written by Mark Twain. To investigate this, we will
consider the proportion of three letter words found in an author’s work.
From eight Twain essays we have:

In [21]:
X = np.array([.225, .262, .217, .240, .230, .229, .235, .217])

From 10 Snodgrass essays we have:

In [22]:
Y = np.array([.209, .205, .196, .210, .202, .207, .224, .223, .220, .201])

1. Perform a Wald test for equality of the means. Use the nonparametric plug-in estimator. Report the p-value and a 95% confidence
interval for the difference of means. What do you conclude? (1.5 points)
2. Now use a permutation test to avoid the use of large sample methods.
What is your conclusion? (Brinegar (1963)) (1.5 points)
3. Assume that samples do indeed come from different populations. Additionally, observed sample means and variaces for the two samples are equal to the true values for the respective population. Estimate the power of the two tests above under two model distributions for the data: Normal and [Beta](https://en.wikipedia.org/wiki/Beta_distribution). Use the same family for both samples (2 points)

#### Q 4.1

In [23]:
def wald_test_for_means(sample1, sample2):
    mean = np.mean(sample1) - np.mean(sample2)
    se = np.sqrt(np.var(sample1) / len(sample1) + np.var(sample2) / len(sample2))
    statistic = np.abs(mean / se)
    pvalue = 2 * sps.norm.cdf(-np.abs(statistic))
    return statistic, pvalue

statistic, pvalue = wald_test_for_means(X, Y)

f"Got statistic = {statistic:.3f} with p-value = {pvalue}"

'Got statistic = 3.945 with p-value = 7.9926649561458e-05'

In [24]:
se = np.sqrt(np.var(X) / len(X) + np.var(Y) / len(Y))
CI1 = np.mean(X)-np.mean(Y) - 1.96*se
CI2 = np.mean(X)-np.mean(Y) + 1.96*se
print ("95% Confidence Interval is (",CI1, CI2, ")")

95% Confidence Interval is ( 0.011156701425582985 0.03319329857441702 )


Reject the hypothesis that means are equal. 95% confident that the difference in the above mentioned interval.

#### Q4.2

In [25]:
def approximate_permutation_criteria(func, sample1, sample2, nruns):
    l1 = len(sample1)
    l2 = len(sample2)
    
    t_obs = func(sample1, sample2)
    sample_joint = np.r_[sample1, sample2]
    counter = 0.
    for r in range(nruns):
        sample_joint = np.random.permutation(sample_joint)
        t = func(sample_joint[:l1], sample_joint[l1:])
        counter += t > t_obs    
    return counter / nruns

In [26]:
T = lambda sample1, sample2: np.mean(sample1) - np.mean (sample2)

pvalue = approximate_permutation_criteria(T, X, Y, nruns=10000)
pvalue

0.0003

P-value of the permutation test is much bigger than was before.